In [ ]:
import os
import numpy as np
import pandas as pd
import json
import itertools
import pickle
import matplotlib.pyplot as plt
import torch
from sentence_transformers import SentenceTransformer, util

## Load data

In [ ]:
# Local data source path
text_source_path = '' 
var_source_path = ''

In [ ]:
text = pd.read_csv(os.path.join(text_source_path, 'g2022_adid_text.csv.gz'))
var = pd.read_csv(os.path.join(var_source_path, 'g2022_adid_var.csv.gz'), usecols=['ad_id', 'wmp_creative_id'])

In [ ]:
df = var.merge(text, on='ad_id', how='left').drop_duplicates()

In [ ]:
unique = df[['wmp_creative_id']].drop_duplicates(subset=['wmp_creative_id'], keep='last').reset_index(drop=True)

In [ ]:
unique.to_csv('../input_data/unique_creative_id_index_mapping.csv', index=False)

In [ ]:
'''
Mapping between corpus embedding index and creative ID 
'''
unique = pd.read_csv('../input_data/unique_creative_id_index_mapping.csv')

## Load original model  (Skip this section to load the reordered embeddings)

In [ ]:
corpus_embeddings0 = torch.load('../model/corpus_embedding_google2022_unique_lite.pt', map_location=torch.device('cpu'))

In [ ]:
corpus_embeddings0.shape

## reorder corpus embedding indices to match cid ordering

**This would make the pairwise similarity computation much faster**

(Because pandas indexing is slow)

In [ ]:
unique.loc[:, 'cid_index'] = unique.wmp_creative_id.apply(lambda x: int(x.lstrip('cid_')))

In [ ]:
len(unique.cid_index.unique())

In [ ]:
reordered = torch.zeros(corpus_embeddings.shape[0], corpus_embeddings.shape[1], dtype=corpus_embeddings.dtype)

In [ ]:
assert reordered.shape == corpus_embeddings.shape

In [ ]:
indices = unique.cid_index.tolist()

indices = torch.tensor([indices for i in range(corpus_embeddings.shape[1])])

In [ ]:
indices = indices.T

In [ ]:
torch.save(reordered, 'model/corpus_embedding_google2022_unique_lite_reordered.pt') 

## Import the reordered embeddings

In [ ]:
'''
This model should be loaded for google data analysis: calculation performs much faster
'''
corpus_embeddings = torch.load('model/corpus_embedding_google2022_unique_lite_reordered.pt')